# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,deir ez-zor,35.3359,40.1408,31.29,33,0,7.93,SY,1690082644
1,1,estevan,49.1334,-102.9842,22.34,56,0,2.06,CA,1690082646
2,2,grytviken,-54.2811,-36.5092,-4.52,80,21,4.20,GS,1690082647
3,3,nogliki,51.8333,143.1667,23.84,59,81,3.99,RU,1690082647
4,4,georgetown,5.4112,100.3354,31.24,86,40,3.60,MY,1690082589


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    size="Humidity",
    color = "City",
    frame_width =800,
    frame_height =600,
    alpha= 0.5
    )

# Display the map
display(map_plot_city)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_city= city_data_df[(city_data_df["Max Temp"]>20) & (city_data_df["Max Temp"]<24) & (city_data_df["Cloudiness"]==0) & (city_data_df["Wind Speed"]<4.5)]

# Drop any rows with null values
ideal_weather_city=ideal_weather_city.dropna()

# Display sample data
ideal_weather_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,estevan,49.1334,-102.9842,22.34,56,0,2.06,CA,1690082646
25,25,conceicao do araguaia,-8.2578,-49.2647,21.08,39,0,1.12,BR,1690082610
27,27,bethel,41.3712,-73.4140,22.17,82,0,2.06,US,1690082412
35,35,gurupi,-11.7292,-49.0686,22.91,45,0,2.72,BR,1690082674
51,51,tahlequah,35.9154,-94.9700,22.27,94,0,0.00,US,1690082679
193,193,hamilton,39.1834,-84.5333,23.21,75,0,3.09,US,1690082706
200,200,sinop,41.7711,34.8709,20.19,48,0,2.42,TR,1690082725
217,217,nova olimpia,-14.7972,-57.2881,20.53,37,0,2.01,BR,1690082730
257,257,sao vicente,-23.9631,-46.3919,21.10,26,0,1.39,BR,1690082741
321,321,international falls,48.6011,-93.4110,20.02,74,0,3.09,US,1690082758


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city.loc[:,["City","Country", 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,estevan,CA,49.1334,-102.9842,56,
25,conceicao do araguaia,BR,-8.2578,-49.2647,39,
27,bethel,US,41.3712,-73.4140,82,
35,gurupi,BR,-11.7292,-49.0686,45,
51,tahlequah,US,35.9154,-94.9700,94,
193,hamilton,US,39.1834,-84.5333,75,
200,sinop,TR,41.7711,34.8709,48,
217,nova olimpia,BR,-14.7972,-57.2881,37,
257,sao vicente,BR,-23.9631,-46.3919,26,
321,international falls,US,48.6011,-93.4110,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "categories": "accommodation.hotel",
        "radius": radius,
        "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")
print("---------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
 
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


Starting hotel search
---------------------
estevan - nearest hotel: Days Inn
conceicao do araguaia - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
gurupi - nearest hotel: Via Norte
tahlequah - nearest hotel: Days Inn & Suites by Wyndham Tahlequah
hamilton - nearest hotel: North Vista Manor
sinop - nearest hotel: No hotel found
nova olimpia - nearest hotel: Olímpia Hotel
sao vicente - nearest hotel: Pousada Vitória
international falls - nearest hotel: Copper River Inn & Conference Centre
puerto maldonado - nearest hotel: Puerto Palmeras
florence - nearest hotel: Old bridge Apartment
yarmouth - nearest hotel: The Inn At Cape Cod
podgorica - nearest hotel: Kerber
westport - nearest hotel: Norwalk Inn and Conference Center
oskaloosa - nearest hotel: Fairfield Inn & Suites by Marriott Oskaloosa
laurel - nearest hotel: Quality Inn & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_city = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    hover_cols=["Hotel Name", "Country"],
    size="Humidity",
    color = "City",
    frame_width =800,
    frame_height =600,
    alpha= 0.8
    )

# Display the map
display(map_plot_city)


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)